In [ ]:
import os
import cv2
import numpy as np

def preprocess_image(image):
    # Step 2: Convert RGB image to YCbCr color space
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Step 3: Detect skin/non-skin by chrominance component Cb and Cr
    cb = ycbcr_image[:,:,1]
    cr = ycbcr_image[:,:,2]
    skin_mask = cv2.inRange(ycbcr_image, (0, 133, 77), (255, 173, 127))  # Skin color range
    
    # Step 4: Convert the image from the previous step to a binary image
    binary_image = cv2.threshold(skin_mask, 0, 255, cv2.THRESH_BINARY)[1]
    
    # Step 5: Morphological operation (closing) is used to fill gaps in small regions and remove unimportant information
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    
    # Step 6: Convert image to RGB image to obtain an ear color image
    ear_color_image = cv2.bitwise_and(image, image, mask=closing)
    
    return ear_color_image

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            # Read the image using OpenCV
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
    return images

# Define the input and output directories
input_folder = 'D:\Prayag Files\TIET\Extras\Ear Detection\subset-1'
output_folder = 'D:\Prayag Files\TIET\Extras\Ear Detection\output images'

# Create the output directory if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load images from the input folder
input_images = load_images_from_folder(input_folder)

# Process and save images
for img in input_images:
    output_img = preprocess_image(img)
    filename = os.path.basename(img)
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, output_img)
